In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

# TODO: comment out if you want to check info logs
# import logging
# logging.basicConfig(level=logging.INFO)
import joblib
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

from airline_delay.data.make_datasets import make_airline_dataframe
from airline_delay.data.prep_datasets import prepare_regression_tabular_data
from airline_delay.features.skl_build_features import FeatureNames, make_data_transformer
from airline_delay.models.skl_train_models import (
    regression_evaluation,
    print_regression_scoring,
    train_and_evaluate,
)

In [2]:
df = make_airline_dataframe(
    Path('../data/flights_sample_3m.csv')
)

In [3]:
predictors = list(df.columns)
# do not include _DELAY columsn because these only have values when we know that the plane has been delayed.
predictors = [col for col in predictors if "DELAY_" not in col]
predictors.remove('DEP_DELAY')
predicted = 'DEP_DELAY'

In [4]:
dataset = prepare_regression_tabular_data(
    df, predictors, predicted
)

/Users/najibmozanem/Library/CloudStorage/OneDrive-McGillUniversity/AC/IVADO workshop/REPOS/bank-marketing-cohort-2/src/airline_delay/data/prep_datasets.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[col] = data_frame[col].fillna(data_frame[col].median())
/Users/najibmozanem/Library/CloudStorage/OneDrive-McGillUniversity/AC/IVADO workshop/REPOS/bank-marketing-cohort-2/src/airline_delay/data/prep_datasets.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[col] = data_frame[col]

In [5]:
numerical_cols = dataset.train_x.select_dtypes(exclude='object').columns.tolist()
categorical_cols = dataset.train_x.select_dtypes('object').columns.tolist()
feature_names = FeatureNames(numerical_cols, categorical_cols)
input_transformer = make_data_transformer(feature_names)
regressor = DecisionTreeRegressor()

In [15]:
model = Pipeline(
    [
        ('input_transformer', input_transformer),
        ('regressor', regressor),
    ]
)
model.fit(dataset.train_x, dataset.train_y)
results = regression_evaluation(model, dataset)
print_regression_scoring(results)

+-------+--------+-------+-------+
| Split |  MSE   |  MAE  |   R2  |
+-------+--------+-------+-------+
| train |  0.0   |  0.0  |  1.0  |
|  val  | 30.709 | 1.969 | 0.988 |
|  test | 36.335 | 1.982 | 0.985 |
+-------+--------+-------+-------+
